In [37]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import matplotlib
pesos = matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ','))
dates = matplotlib.dates.DateFormatter('%b %d, %Y')
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15
plt.style.use('seaborn-whitegrid')
import plotly.graph_objects as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
pd.set_option('display.float_format', '{:.0f}'.format)
pd.options.display.max_rows = 100

In [2]:
path = 'D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/'

##### Get holidays

In [290]:
HP = pd.read_csv(path+'Actual/Holidays_ALL.csv',index_col='ds',parse_dates=['ds'])
HP['event']  = HP.holiday.apply(lambda x: 'Payroll' if 'Payroll' in x else 'Holiday')
HP.reset_index(inplace=True)
HP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ds            451 non-null    datetime64[ns]
 1   holiday       451 non-null    object        
 2   lower_window  451 non-null    int64         
 3   upper_window  451 non-null    int64         
 4   event         451 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 17.7+ KB


In [305]:
holi = HP.pivot_table(index='ds',columns='event',values='holiday',aggfunc='count').reset_index()
holi['ds_next'] = holi.ds+pd.to_timedelta('1 days')
holi['event'] = holi[['Holiday','Payroll']].fillna(0).apply(lambda x: "Holiday and Payroll" if sum(x) == 2 else "")

holi[holi[['Holiday','Payroll']].isnull().sum(axis=1)==2]
holi.loc[holi.Holiday.isnull(),'event']='Payroll'
holi.loc[holi.Payroll.isnull(),'event']='Holiday'

##### get atm profile for tid name 

In [273]:
atm_profile = pd.read_excel(path+'Actual/ATM Profile.xlsx', dtype={'TID':'str'})

##### get data atm = '91018953'

In [438]:
atm = '91018953'
data_all = pd.read_csv(path+'Prophet Forecast All/_'+atm+'_.csv'
                   ,dtype={'tid':str},parse_dates=['ds'],index_col='ds')
data_all['error'] = data_all.y-data_all.yhat
data_all['tid_name'] = atm_profile[(atm_profile.TID==atm) | (atm_profile['Branch Code Orig']==atm)]['ATM Name'].values[0]
data_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5658 entries, 2017-01-01 to 2019-12-31
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   y          5658 non-null   float64
 1   yhat       5658 non-null   float64
 2   pred_date  5658 non-null   object 
 3   pred_day   5658 non-null   object 
 4   se         5658 non-null   float64
 5   tid        5658 non-null   object 
 6   error      5658 non-null   float64
 7   tid_name   5658 non-null   object 
dtypes: float64(4), object(4)
memory usage: 397.8+ KB


In [439]:
data_all

,y,yhat,pred_date,pred_day,se,tid,error,tid_name
ds,,,,,,,,
2017-01-01,0,0,2018-12-31,day 0,0,91018953,0,EPSON LIPA OFF 1_B
2017-01-02,8900,212794,2018-12-31,day 1,41572696326,91018953,-203894,EPSON LIPA OFF 1_B
2017-01-03,61800,267051,2018-12-31,day 2,42127788917,91018953,-205251,EPSON LIPA OFF 1_B
2017-01-04,197000,300820,2018-12-31,day 3,10778678629,91018953,-103820,EPSON LIPA OFF 1_B
2017-01-05,719400,343527,2018-12-31,day 4,141280785723,91018953,375873,EPSON LIPA OFF 1_B
...,...,...,...,...,...,...,...,...
2019-12-27,0,1500771,2019-12-17,day 9,2252314369036,91018953,-1500771,EPSON LIPA OFF 1_B
2019-12-28,0,449222,2019-12-17,day 10,201800308346,91018953,-449222,EPSON LIPA OFF 1_B
2019-12-29,0,0,2019-12-17,day 11,0,91018953,0,EPSON LIPA OFF 1_B


##### get model performance

In [440]:
data_all['dev_rmse'] = np.sqrt(data_all[data_all.index<'2019-01-01'].se.mean())
data_all['val_rmse'] = data_all[data_all.index>='2019-01-01'].groupby('pred_day').se.mean().agg(np.sqrt).mean()
data_all['%error'] = (data_all.error/data_all.y).fillna(0).abs()
data_all['dev_mape'] = data_all[(data_all['%error'] != np.inf) & (data_all.index<'2019-01-01')]['%error'].mean()
data_all['val_mape'] = data_all[(data_all['%error'] != np.inf) & (data_all.index>='2019-01-01')]['%error'].mean()

##### format to required 

In [441]:
data = data_all.copy()

In [442]:
dev = data[data.index<'2019-01-01'].copy()
val = data[(data.index>='2019-01-01')].copy()
val[['upper','lower','median']] = val[['yhat','pred_day']].pivot_table(index=val.index,columns='pred_day',values='yhat').agg(['max','min','median'],axis=1)
all_ = pd.concat([dev,val[val.pred_day=='day 0']])
all_.upper = all_.upper.fillna(all_.yhat)
all_.lower = all_.lower.fillna(all_.yhat)

In [443]:
holidays = holi[holi.ds.isin(all_.index)]

# plot

In [444]:
def flag_holi(holi):
    shape =  [
#         dict(
#             type="rect",
#             # x-reference is assigned to the x-values
#             xref="x",
#             # y-reference is assigned to the plot paper [0,1]
#             yref="paper",
#             x0=holi.ds[i].date(),
#             y0=0,
#             x1=holi.ds_next[i].date(),
#             y1=1,
#             fillcolor="LightSalmon",
#             opacity=0.5,
#             layer="below",
#             line_width=0)
#      for i in holi[holi.ds.isin(all_.index) & (holi.event=='Payroll')].index] + \
#     [
        dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=holi.ds[i].date(),
            y0=0,
            x1=holi.ds_next[i].date(),
            y1=1,
            fillcolor="LightCoral",
            opacity=0.5,
            layer="below",
            line_width=0)
     for i in holi[holi.ds.isin(all_.index) & (holi.event=='Holiday')].index]
    return shape

def get_bounds(all_):
    bounds = ((all_[['y','error']].agg(['max','min'])/1000000).round())*1000000
    bounds.y = bounds.y+1000000
    bounds.error = bounds.error-1000000
    return bounds

In [445]:
# fig, ax = plt.subplots(figsize = (20,10))
# ax.plot(all_.index,all_.y,'ko-',label='Actual Withdrawals')
# ax.plot(all_.index,all_.yhat,'go-',label='One-Day Ahead Forecast')
# ax.bar(all_.index,all_.error,color = 'gray',alpha=0.2,label = 'Forecast Error')
# ax.fill_between(all_.index,all_.lower,all_.upper,color='green',alpha=0.8,label='14-Day Forecast Range')
# ax.set_ylabel('Pesos',fontsize=20)
# ax.set_xlabel('Date',fontsize=20)
# ax.get_yaxis().set_major_formatter(pesos)
# ax.get_xaxis().set_major_formatter(dates)
# # flag_holi()
# plt.yticks(np.arange(-1000000,3000000,500000))
# plt.grid(which='both',axis='both',linestyle='--')
# plt.legend(fontsize = 13)

In [446]:
def visual(all_,holi, path):
    atm =  all_.tid[0]
    atm_name = all_.tid_name[0]
    val_mape = "%{:.2f}".format(all_.val_mape[0]*100)
    val_rmse = "Php {:.2f} K".format(all_.val_rmse[0]/1000)
    pred = list(all_.yhat.values)
    pred_upper = list(all_.upper)
    pred_lower = list(all_.lower)
    error = list(all_.error)

    test_dates_rev = list(all_.index)[::-1]
    test_dates = list(all_.index)
    test = list(all_.y.values)

    # Layout
    layout = go.Layout(
        paper_bgcolor='rgb(255,255,255)',
        plot_bgcolor='rgb(255,255,255)',
        yaxis=dict(
            gridcolor='rgb(165,172,175)',
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(127,127,127)',
            ticks='outside',
            zeroline=False
        ),
        xaxis=dict(
            gridcolor='rgb(165,172,175)',
            showgrid=True,
            gridwidth = 0.5,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(127,127,127)',
            ticks='outside',
            zeroline=False,
            rangeselector=dict(
                buttons=list([
                    dict(count=7,
                        label="1w",
                        step="day"),
                    dict(count=1,
                         label="1m",
                         step="month"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=False
            ),
            type = "date"
        ),
        xaxis_range=['2019-02-01','2019-04-30'],
        xaxis_tickmode = 'array',
        xaxis_tickvals = [15*k for k in range(len(test_dates))],
        xaxis_ticktext = [],
        showlegend=True
    )

    trace1 = go.Scatter(
        x =test_dates,
        y =test,
        line = dict(color='rgba(0,0,0,1)'),
        mode = 'lines+markers',
        showlegend = True,
        name = 'Actual Withdrawal')

    upper_bound = go.Scatter(
        name = 'Upper Bound',
        x =test_dates,
        y = pred_upper,
        mode = 'lines',
        marker = dict(color = 'rgba(44,160,44,0.2)'),
        line = dict(width = 0),
        fillcolor = 'rgba(44,160,44,0.2)',
        fill ='tonexty',
        showlegend = False)

    trace2 = go.Scatter(
        x =test_dates,
        y = pred,
        line=dict(color='green'),
        mode = 'lines+markers',
        showlegend=True,
        name='Forecasted Withdrawal',
        fillcolor = 'rgba(44,160,44,0.2)',
        fill = 'tonexty')

    lower_bound = go.Scatter(
        name='Lower Bound',
        x=test_dates,
        y=pred_lower,
        marker = dict(color = 'rgba(44,160,44,0.2)'),
        line = dict(width = 0),
        mode = 'lines',
        showlegend = False)

    trace3 = go.Bar(
        x = test_dates,
        y = error,
        marker_color = 'rgba(207,207,207,1)',
        showlegend = True,
        name = 'Forecast Error')
    forecasted = [lower_bound,trace2,upper_bound]

    # Create figure
    fig = go.Figure(data = forecasted,layout = layout)

    # Add Trace
    fig.add_trace(trace1)
    fig.add_trace(trace3)

    # Add range slider
    fig.update_layout(
        height = 600,
        width = 1200,
        xaxis_title = 'Date',
        yaxis_title = 'Daily Withdrawal',
        xaxis_tickformat = '%b %d <br>%Y <br>(%a)',
        title = 'Actual vs. Predicted Daily Withdrawals (ATM ' + atm + ': ' + atm_name + ')'+'<br>'+ \
                'Ave Daily Error: ' + val_rmse + ' (' + val_mape + ')',
        shapes=flag_holi(holi)
    )

    bounds = get_bounds(all_)
    fig.update_yaxes(range = [bounds.error['min'],bounds.y['max']],
                     showline = True,
                     linecolor = 'gray',
                     mirror = True,
                     tickprefix = '₱')

#     fig.update_xaxes(range = [pd.to_datetime('01-31-2019'),pd.to_datetime('05-01-2019')],
#                      showline = True,
#                      linecolor = 'gray',
#                      mirror = True,
#     )

    fig.show()
    # fig.write_image(validation+'ATM8739.png')
    plotly.offline.plot(fig, filename=path + '_'+atm+'_.html', auto_open=False)
    return fig

In [447]:
visual(all_,holidays,path+'Prophet Visuals/')